In [1]:
import supereeg as se
import numpy as np
import pandas as pd
import os
import glob as glob
import copy
from scipy.io import loadmat as load
from scipy.spatial.distance import pdist, cdist, squareform


/Users/lucyowen/.pyenv/versions/3.7.2/envs/my-data-project/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [3]:
def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, n)[:n]
    indices = indices[np.argsort(flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [4]:
def rbf(centers, widths, locs):
    """
    Radial basis function
    Parameters
    ----------
    centers : ndarray
        rbf coordinates (one row per RBF)
    widths : ndarray
        RBF radii
    locs : ndarray
        locations to evaluate the RBFs (one row per location)
        
    Returns
    ----------
    results : ndarray
        Matrix of RBF weights for each RBF (row), at each location (column)
    """    
    weights = np.exp(np.divide(-cdist(locs, centers, metric='euclidean') ** 2, np.tile(np.array(widths, ndmin=2), [locs.shape[0], 1])))
    return weights.T

In [5]:
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [6]:
results_dir = os.path.join(fileDir, '../../data/', 'results')

In [7]:
data_dir = os.path.join(results_dir, 'mean_corrs')

In [8]:
replication_dir ='../figs/replication/'
if not os.path.exists(replication_dir):
    os.mkdir(replication_dir)

In [9]:
#neurosynth_dir ='../figs/neurosynth_data_/'
neurosynth_dir = os.path.join(replication_dir, 'neurosynth_data')
if not os.path.exists(neurosynth_dir):
    os.mkdir(neurosynth_dir)

In [10]:
bo_dir = os.path.join(neurosynth_dir, 'bos')
if not os.path.exists(bo_dir):
    os.mkdir(bo_dir)

In [11]:
posterior = load(os.path.join(fileDir, '../../data/pieman_posterior_K700.mat'))
centers = posterior['posterior']['centers'][0][0][0][0][0]
widths = np.array(list(posterior['posterior']['widths'][0][0][0][0][0][:, 0].T))

In [12]:
template = se.helpers._gray(res=2)

In [13]:
levels = np.arange(0,15,1)
conditions = ['intact', 'paragraph', 'rest', 'word']

In [14]:
levels

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [15]:
top_n = 10

In [16]:
full_data = pd.DataFrame()

for l in levels:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = largest_indices(np.triu(np.abs(networks)), top_n)
        net_inds = np.concatenate((net_inds[0], net_inds[1]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        
        data = b.get_data().values.ravel()
        xyz = b.get_locs().values
        xyz = xyz[data>.999]
        data = data[data>.999]
        data = np.ones(data.shape)+l
        
        try:
            bo = se.Brain(data=data, locs=xyz)
            outfile = c+ '_' + str(l+1)
            bo.save(os.path.join(bo_dir, outfile + '_largest_abs'))
        except:
            print('cannot save bo ' + outfile)

        np.save(os.path.join(bo_dir,f'{c}_{l + 1}_data_largest_abs.npy'), data)
        np.save(os.path.join(bo_dir,f'{c}_{l + 1}_locs_largest_abs.npy'), xyz)
        
        part_data = pd.DataFrame()
        part_data['X'] = xyz[:, 0]
        part_data['Y'] = xyz[:, 1]
        part_data['Z'] = xyz[:, 2]
        part_data['Data'] = data
        part_data['Condition'] = c
        part_data['Level'] = l + 1
        
        if full_data.empty:
            full_data = part_data
        else:
            full_data = full_data.append(part_data)
        

cannot save bo rest_5
cannot save bo intact_9
cannot save bo intact_11
cannot save bo rest_12


In [17]:
full_data.to_excel(os.path.join(data_dir, 'bo_for_plots.xlsx'), sheet_name='bos')

In [18]:
posteriors_data = pd.DataFrame()
posteriors_data['Centers_X'] = centers[:, 0]
posteriors_data['Centers_Y'] = centers[:, 1]
posteriors_data['Centers_Z'] = centers[:, 2]
posteriors_data['Widths'] = widths

posteriors_data.to_excel(os.path.join(data_dir, 'posteriors.xlsx'), sheet_name='posteriors')